In [ ]:
from src.helpers import get_frames_from_video_file, to_gif

#frames = get_frames_from_video_file("data/vids/real/10 Times Keanu Reeves BROKE the Internet.mp4", 100)
#to_gif(frames)

In [ ]:
# Batch size * number of frames * height * width * channels

In [ ]:
import tensorflow as tf
from pathlib import Path
from src.datagen import FrameGenerator

# DEFINE TRAIN/TEST DIRECTORIES
train_p = Path('data/vids/split/train')
test_p = Path('data/vids/split/test')

# CREATE TRAINING DATASET
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32), tf.TensorSpec(shape = (1), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(train_p, 10, training=True),output_signature = output_signature)

# CREATE TESTING DATASET
test_ds = tf.data.Dataset.from_generator(FrameGenerator(test_p, 10, training=False),output_signature = output_signature)

# VERIFY DATA
train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

test_frames, test_labels = next(iter(test_ds))
print(f'Shape of validation set of frames: {test_frames.shape}')
print(f'Shape of validation labels: {test_labels.shape}')

In [ ]:
from src.models import get_mesonet_model
from src.train import prep_and_train_model

mesonet = get_mesonet_model()

history, trained_model = prep_and_train_model(
    model=mesonet,
    optim="adam",
    train_data=train_ds,
    test_data=test_ds,
    epochs=1,
    batch_size=10
)

In [ ]:
import matplotlib.pyplot as plt

# NOTE: ONLY WORKS FOR MULTIPLE EPOCHS

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from src.models import get_mouthnet_model
from src.train import prep_and_train_model

#####################################################
# WARNING - THIS MODEL HAS CONSIDERABLY MORE PARAMETERS
# IT WILL TAKE WAY LONGER TO TRAIN
# TRY IMPLEMENTING PARALLEL WORKERS
#####################################################

mouthnet = get_mouthnet_model()

history, trained_model = prep_and_train_model(
    model=mouthnet,
    optim="sgd",
    train_data=train_ds,
    test_data=test_ds,
    epochs=1,
    batch_size=10
)